In [1]:
using WGMODES
using Plots

# 定数の定義（単位をμmに変更）
const c = 3e8 # 真空中の光速 [m/s]
const mu0 = 4π * 1e-7 # 真空の透磁率 [H/m]
const epsilon0 = 1 / (mu0 * c^2) # 真空の誘電率 [F/m]

lambda = 1.55 # 波長 [μm]
dx = dy = 0.2 # 空間ステップ [μm] -> メモリ削減のためステップを大きく変更
x = -2:dx:2
y = -1:dy:1

# 索引分布の定義
index = [y_val < 0 ? 1.5 : (y_val < 0.2 || (y_val < 0.5 && abs(x_val) < 1) ? 3.45 : 1.0) for y_val in y, x_val in x]
epsilon = index .^ 2
guess = maximum(index)
nmodes = 5
boundary = "0000"

# モード数を指定（例として1）
mode = 1

hx, hy, neff = WGMODES.wgmodes(lambda, guess, nmodes, dx, dy, epsilon, boundary);

# 有効屈折率の順序を確認
println("neff の順序: ", neff)
sorted_indices = sortperm(abs.(neff))
println("有効屈折率が小さい順のインデックス: ", sorted_indices)

# 各配列のサイズを確認
println("x size: ", size(x))
println("y size: ", size(y))
println("index size: ", size(index))
println("epsilon size: ", size(epsilon))
println("hx size: ", size(hx))
println("hy size: ", size(hy))
println("neff size: ", size(neff))

# メッシュを広げた後の x, y のサイズ確認
x = [x; x[end] + dx]
y = [y; y[end] + dy]
println("Expanded x size: ", size(x))
println("Expanded y size: ", size(y))

# 電界成分の計算
# 有効屈折率 neff[mode] と誘電率 epsilon を用いて E_x と E_y を計算
# E_x = (neff / (c * epsilon0 * epsilon)) * H_y
# E_y = - (neff / (c * epsilon0 * epsilon)) * H_x
ex = (neff[mode] / (c * epsilon0 * index)) .* hy[:, :, mode]
ey = -(neff[mode] / (c * epsilon0 * index)) .* hx[:, :, mode]

# プロット
Plots.plot(
    Plots.heatmap(x, y, index,
        title="Waveguide Index Structure",
        aspectratio=1,
    ),
    Plots.heatmap(x, y, real.(hx[:, :, mode] .+ hy[:, :, mode]),
        title="Waveguide Mode (Total H Field)",
        aspectratio=1,
    ),
    Plots.heatmap(x, y, real.(hx[:, :, mode]),
        title="Waveguide Mode (H_x)",
        aspectratio=1,
    ),
    Plots.heatmap(x, y, real.(hy[:, :, mode]),
        title="Waveguide Mode (H_y)",
        aspectratio=1,
    ),
    Plots.heatmap(x, y, real.(ex),
        title="Waveguide Mode (E_x)",
        aspectratio=1,
    ),
    Plots.heatmap(x, y, real.(ey),
        title="Waveguide Mode (E_y)",
        aspectratio=1,
    ),
    size=(1200, 800),
    layout=(2, 3)
)


neff の順序: ComplexF64[3.2472878567803414 + 0.0im, 3.204872108315866 + 0.0im, 3.1676870758137934 + 0.0im, 3.129180524539248 + 0.0im, 3.041370595818929 + 0.0im]
有効屈折率が小さい順のインデックス: [5, 4, 3, 2, 1]
x size: (21,)
y size: (11,)
index size: (11, 21)
epsilon size: (11, 21)
hx size: (12, 22, 5)
hy size: (12, 22, 5)
neff size: (5,)
Expanded x size: (22,)
Expanded y size: (12,)


MethodError: MethodError: no method matching /(::ComplexF64, ::Matrix{Float64})

Closest candidates are:
  /(::ComplexF64, !Matched::ComplexF64)
   @ Base complex.jl:394
  /(!Matched::LinearAlgebra.Transpose{T, <:AbstractVector} where T, ::AbstractMatrix)
   @ LinearAlgebra ~/.julia/juliaup/julia-1.10.5+0.x64.linux.gnu/share/julia/stdlib/v1.10/LinearAlgebra/src/adjtrans.jl:492
  /(!Matched::LinearAlgebra.Adjoint{T, <:AbstractVector} where T, ::AbstractMatrix)
   @ LinearAlgebra ~/.julia/juliaup/julia-1.10.5+0.x64.linux.gnu/share/julia/stdlib/v1.10/LinearAlgebra/src/adjtrans.jl:491
  ...
